In [6]:
# Inspect cells lines to find what needs to be quantified

In [1]:
import genotations.ensembl
import polars as pl
from pycomfort.files import *

/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys

base = Path("..")
local = (base / "dashboard").resolve()
if local.exists():
    sys.path.insert(0, Path("..").absolute().as_posix())
    sys.path.insert(0, local)
    print(sys.path)
else:
    base = Path(".")
%load_ext autoreload
%autoreload 2

[PosixPath('/home/antonkulaga/sources/expressions-dashboard/dashboard'), '/home/antonkulaga/sources/expressions-dashboard/notebooks/..', '/home/antonkulaga/sources/expressions-dashboard/notebooks', '/home/antonkulaga/sources/expressions-dashboard', '/home/antonkulaga/micromamba/envs/dashboard/lib/python310.zip', '/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10', '/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/site-packages', '/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10/site-packages']


In [3]:
pl.Config.set_tbl_width_chars(10000)
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_rows(20)

polars.cfg.Config

In [4]:
from dashboard.cell_models import *
from dashboard.config import *

In [5]:
locations = Locations(base)
locations.cell_lines_cache

PosixPath('../data/inputs/cell_lines/CACHE')

In [7]:
cells = Cells(locations.cell_lines)

cell line /home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/Glu-Tag does not seem to exist!
cell line /home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/INS-1 does not seem to exist!


In [9]:
cells.species_expressions_by_name.keys()

dict_keys(['Homo_sapiens'])

In [89]:
print(cells.folder.absolute())
positive_quants: OrderedDict = OrderedDict()
negative_quants: OrderedDict = OrderedDict()
empty_quants: list = []
for line in dirs(cells.folder):
   samples = dirs(line)
   positive = dirs(line).flat_map(lambda s:
                              dirs(s).filter(
                                  lambda r: dirs(r).exists(lambda d: d.name.startswith("quant"))
                              ).map(lambda r: r.name)).to_list()
   negative = dirs(line).flat_map(lambda s:
                              dirs(s).filter(
                                  lambda r: not dirs(r).exists(lambda d: d.name.startswith("quant"))
                              ).map(lambda r: r.name)).to_list()
   if len(positive) > 0:
       positive_quants[line.name] = positive
   if len(negative) > 0:
       negative_quants[line.name] = negative
   if len(positive) ==0 and len(negative) == 0:
       empty_quants.append(line.name)

print("QUANTS: ")
print(list(positive_quants.keys()))
print("NO QUANTS: ")
print(list(negative_quants.keys()))
print("EMPTY")
print(empty_quants)


/home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines
QUANTS: 
['SKOV3', 'EA.hy926', 'PANC-1', 'MelJuSo', 'HEK293T', 'Jurkat', 'CCD-1070Sk', 'THP-1', 'A549', 'RAW_264.7', 'A375', 'B16F10']
NO QUANTS: 
['MDBK', 'INS1', 'B16F1', 'Glu-Tag', 'B16F10']
EMPTY
['SkMel-23', 'HepaRG', 'CACHE', 'Me275', 'BV2', 'COLO205', 'MNT-1', 'RAW']


In [25]:
tprint(cells.folder, 1)

cell_lines
	.gitignore
	cell_lines_toc.tsv
	MDBK
	SKOV3
	HepaRG
	EA.hy926
	PANC-1
	INS1
	CACHE
		.gitignore
		Homo_sapiens_gene_names.parquet
		Homo_sapiens_transcript_expressions.parquet
		Homo_sapiens_transcript_expressions_with_exons.parquet
		Homo_sapiens_gene_expressions.parquet
	MelJuSo
	HEK293T
	Jurkat
	B16F1
	RAW 264.7
	CCD-1070Sk
	THP-1
	A549
	RAW_264.7
	B16F10
	RAW


In [90]:
non_human_lines = cells.toc_samples.filter(pl.col("Species") != "Human").select("Cell line name").to_series().to_list()
for l in non_human_lines:
    line = (cells.folder / l)
    if not line.exists():
        print(line)


/home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/INS-1
/home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/PANC-1 
/home/antonkulaga/sources/expressions-dashboard/data/inputs/cell_lines/RAW 264.7


In [91]:
non_human_lines

['Glu-Tag',
 'INS-1',
 'PANC-1 ',
 'CCD-1070Sk',
 'RAW 264.7',
 'B16F10',
 'B16F1',
 'MDBK']

##